### Create DeLta Table

In [0]:
%sql
CREATE OR REPLACE TABLE employee_delta_upd(
          Emp_id INT,
          Emp_name STRING,
          Gender STRING,
          Salary INT, 
          Dept STRING)
        USING DELTA
        LOCATION '/FileStore/tables/delta3/createtable'


### Populate Sample Data

In [0]:
%sql
INSERT INTO employee_delta_upd VALUES
(100, 'Sabrina', 'F', 10000, 'Music'),
(101, 'Emily', 'F', 9500, 'Finance'),
(102, 'Michael', 'M', 12000, 'IT'),
(103, 'Olivia', 'F', 11000, 'Marketing'),
(104, 'James', 'M', 10500, 'Sales'),
(105, 'Sophia', 'F', 9800, 'HR'),
(106, 'William', 'M', 11500, 'Operations');

num_affected_rows,num_inserted_rows
7,7


### Method 1: SQL Standard using Table Name

In [0]:
%sql
UPDATE employee_delta_upd SET Salary=16000 WHERE Emp_name='Sabrina'

num_affected_rows
1


In [0]:
%sql
SELECT * FROM employee_delta_upd WHERE Emp_name='Sabrina'

Emp_id,Emp_name,Gender,Salary,Dept
100,Sabrina,F,16000,Music


### Method 2: SQL Standard using Table Path

In [0]:
%sql
SELECT * FROM employee_delta_upd WHERE Emp_name='Olivia'

Emp_id,Emp_name,Gender,Salary,Dept
103,Olivia,F,11000,Marketing


In [0]:
%sql
UPDATE delta.`/FileStore/tables/delta3/createtable` SET salary=15000  WHERE EMp_name='Olivia'  -- use backticks

num_affected_rows
1


In [0]:
%sql
SELECT * FROM employee_delta_upd WHERE Emp_name='Olivia'

Emp_id,Emp_name,Gender,Salary,Dept
103,Olivia,F,15000,Marketing


### Method 3: PySpark Standard using Table Instance

In [0]:
%sql
SELECT * FROM employee_delta_upd WHERE Emp_name='Emily'

Emp_id,Emp_name,Gender,Salary,Dept
101,Emily,F,9500,Finance


In [0]:
from delta.tables import DeltaTable

deltatable = DeltaTable.forPath(spark, '/FileStore/tables/delta3/createtable')
# Declare the predicate by using the SQL-Formatted String
deltatable.update(
    condition = "Emp_name= 'Emily'",
    set = {"salary": "17500"}
)

In [0]:
%sql
SELECT * FROM employee_delta_upd

Emp_id,Emp_name,Gender,Salary,Dept
100,Sabrina,F,16000,Music
101,Emily,F,17500,Finance
102,Michael,M,12000,IT
103,Olivia,F,15000,Marketing
104,James,M,10500,Sales
105,Sophia,F,9800,HR
106,William,M,11500,Operations


In [0]:
from pyspark.sql.functions import col, lit

# Declare the predicate by using the SQL-Formatted String
deltatable.update(
    condition = col('Emp_name')=='Michael',
    set = {'Dept': lit('Fashion')}
)

In [0]:
%sql
SELECT * FROM employee_delta_upd

Emp_id,Emp_name,Gender,Salary,Dept
100,Sabrina,F,16000,Music
101,Emily,F,17500,Finance
102,Michael,M,12000,Fashion
103,Olivia,F,15000,Marketing
104,James,M,10500,Sales
105,Sophia,F,9800,HR
106,William,M,11500,Operations
